# Instruction Fine-tuning sobre un LLM Base

## 0. Instalación de librerías externas

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=

## 1. Comportamiento de [Flan-T5-small](https://huggingface.co/google/flan-t5-small) sin Fine-tuning

### Lectura del modelo y tokenizador

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Importamos el tokenizador
tokenizer_FT5 = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Importamos el modelo pre-entrenado
model_FT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Lectura de Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/NLP/cls_llm_dataset.csv')

In [ ]:
data.head()

,Text,Label
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria


In [ ]:
data['Text'][0]

'Manchado o sangrado vaginal\r\nDolor o calambres en el abdomen o la parte inferior de la espalda\r\nFluidos o tejidos que salen por la vagina\r\nSi ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.\r\nRecuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.'

In [ ]:
data['Label'][0]

'Aborto espontáneo'

### Generación de texto

In [ ]:
context = data['Text'][0]

In [ ]:
context = data['Text'][1]

In [ ]:
context = data['Text'][2]

In [ ]:
question = '¿Qué enfermedad tengo respecto a estos síntomas?'

In [ ]:
# ("Answer based on context:\n\n{context}\n\n{question}", "{answer}")
prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5.generate(prompt_tokens, max_length=200)

# Transformamos los tokens generados en texto
print(tokenizer_FT5.decode(outputs[0]))

<pad> Manchado o sangrado vaginal Dolor o calambres en el abdomen o la parte inferior de la espalda</s>


## 2. Selección y preparación del conjunto de datos

### Formato del conjunto de datos

Es habitual utilizar plantillas que proponen los desarrolladores de los LLM para diseñar nuestros ejemplos de entrenamiento: https://github.com/google-research/FLAN/blob/main/flan/v2/flan_templates_branched.py

In [ ]:
# prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"
def convert_to_template(context, question):
    return f"Responde basado en contexto:\n\n{context}\n\n{question}"

In [ ]:
data['prompt'] = data.apply(lambda x: convert_to_template(x['Text'], question), axis=1)

In [ ]:
print(data['prompt'][0])

Responde basado en contexto:

Manchado o sangrado vaginal
Dolor o calambres en el abdomen o la parte inferior de la espalda
Fluidos o tejidos que salen por la vagina
Si ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.
Recuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.

¿Qué enfermedad tengo respecto a estos síntomas?


In [ ]:
data.head()

,Text,Label,prompt
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo,Responde basado en contexto:\n\nManchado o san...
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental,Responde basado en contexto:\n\nDolor de muela...
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas,"Responde basado en contexto:\n\nRobar, falsifi..."
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia,Responde basado en contexto:\n\nIncapacidad pa...
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria,Responde basado en contexto:\n\nLos cambios en...


In [ ]:
data_cls = data[['prompt', 'Label']]
data_cls.head()

,prompt,Label
0,Responde basado en contexto:\n\nManchado o san...,Aborto espontáneo
1,Responde basado en contexto:\n\nDolor de muela...,Absceso dental
2,"Responde basado en contexto:\n\nRobar, falsifi...",Abuso de drogas recetadas
3,Responde basado en contexto:\n\nIncapacidad pa...,Acalasia
4,Responde basado en contexto:\n\nLos cambios en...,Acantosis pigmentaria


In [ ]:
# Dividimos el conjunto de datos en entrenamiento,test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data_cls, test_size=0.2, random_state=42)

In [ ]:
# Convertimos a formato DatasetDict
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

ds = DatasetDict()

ds['train'] = train_dataset
ds['test'] = test_dataset

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 882
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 221
    })
})

In [ ]:
ds['train']['prompt'][1]

'Responde basado en contexto:\n\nCrees que tu piel está infectada. Los indicios incluyen fiebre y exudado purulento de las ampollas.\r\nTienes dolor e inflamación en los pulmones, los ojos o las fosas nasales, tal vez por haber inhalado un alérgeno.\r\nPiensas que la erupción cutánea ha dañado el revestimiento mucoso de la boca y el tubo digestivo.\n\n¿Qué enfermedad tengo respecto a estos síntomas?'

In [ ]:
ds['train']['Label'][1]

'Dermatitis de contacto'

In [ ]:
# Reducimos el conjunto de datos
NUM_EJ_TRAIN = 840
NUM_EJ_VAL = 221
NUM_EJ_TEST = 40

# Subconjunto de entrenamiento
ds['train'] = ds['train'].select(range(NUM_EJ_TRAIN))

# Subconjunto de validación
ds['validation'] = ds['train'].select(range(NUM_EJ_VAL))

# Subconjunto de pruebas
ds['test'] = ds['test'].select(range(NUM_EJ_TEST))

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 700
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 221
    })
    validation: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 182
    })
})

In [ ]:
ds['test']['prompt'][1]

'Responde basado en contexto:\n\nOmitir comidas o poner excusas para no comer\r\nAdoptar una dieta vegetariana demasiado restrictiva\r\nCentrarse excesivamente en la alimentación saludable\r\nPrepararse los alimentos, en lugar de comer lo que la familia come\r\nAlejarse de las actividades sociales normales\r\nPreocuparse o quejarse continuamente por estar gordo y hablar sobre cómo bajar de peso\r\nMirarse con frecuencia al espejo para ver los defectos que se perciben\r\nComer reiteradamente grandes cantidades de dulces o de alimentos con alto contenido de grasas\r\nTomar suplementos dietéticos, laxantes o productos herbarios para bajar de peso\r\nEjercitarse en exceso\r\nTener callosidades en los nudillos por provocarse los vómitos\r\nTener problemas de pérdida del esmalte dental, un posible signo de vómitos reiterados\r\nIr al baño durante las comidas\r\nDurante una comida o refrigerio, comer una cantidad mucho mayor de lo que se considera normal\r\nExpresar depresión, enojo, vergüenz

In [ ]:
ds['test']['Label'][1]

'Trastornos de la alimentación'

### 2.3. Tokenización del conjunto de datos

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

Como los algoritmos reciben secuencias del mismo tamaño debemos obtener la secuencia más larga de nuestro conjunto de datos y realizar padding al resto de secuencias para que todas tengan el mismo tamaño.

In [ ]:
from datasets import concatenate_datasets

# Calculamos el tamaño máximo de prompt
prompts_tokens = concatenate_datasets([ds["train"], ds["validation"], ds["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True) # Va a truncar en 512 que es el tamaño máximo para este modelo
max_token_len = max([len(x) for x in prompts_tokens["input_ids"]])
print(f"Maximo tamaño de prompt: {max_token_len}")

# Calculamos el tamaño máximo de completion
completions_tokens = concatenate_datasets([ds["train"], ds["validation"], ds["test"]]).map(lambda x: tokenizer(x["Label"], truncation=True), batched=True)
max_completion_len = max([len(x) for x in completions_tokens["input_ids"]])
print(f"Maximo tamaño de completion: {max_completion_len}")

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

Maximo tamaño de prompt: 512


Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

Maximo tamaño de completion: 34


In [ ]:
def padding_tokenizer(datos):
  # Tokenizar inputs (prompts)
  model_inputs = tokenizer(datos['prompt'], max_length=max_token_len, padding="max_length", truncation=True)

  # Tokenizar labels (completions)
  model_labels = tokenizer(datos['Label'], max_length=max_completion_len, padding="max_length", truncation=True)

  # Sustituimos el caracter de padding de las completion por -100 para que no se tenga en cuenta en el entrenamiento
  model_labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_labels["input_ids"]]

  model_inputs['labels'] = model_labels["input_ids"]

  return model_inputs

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 700
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 221
    })
    validation: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 182
    })
})

In [ ]:
ds_tokens = ds.map(padding_tokenizer, batched=True , remove_columns=['prompt','Label'])
#remove_columns=['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt']

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/221 [00:00<?, ? examples/s]

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

In [ ]:
ds_tokens

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 700
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 221
    })
    validation: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 182
    })
})

In [ ]:
ds_tokens["train"]["input_ids"][10]

[23483,
 15,
 3905,
 9,
 26,
 32,
 3,
 35,
 2625,
 32,
 10,
 325,
 3,
 7,
 2,
 8027,
 159,
 142,
 915,
 9,
 3,
 35,
 3,
 26129,
 7,
 3,
 63,
 10381,
 3,
 7,
 2,
 29,
 235,
 2754,
 3,
 4331,
 2,
 152,
 142,
 122,
 2,
 29,
 50,
 3,
 26129,
 5,
 465,
 3,
 28626,
 1841,
 6,
 50,
 7,
 3,
 26129,
 7,
 4353,
 15,
 537,
 576,
 11542,
 12594,
 3,
 63,
 10381,
 3,
 7,
 2,
 29,
 235,
 2754,
 150,
 108,
 25569,
 11693,
 15,
 537,
 3,
 35,
 3,
 15,
 40,
 1817,
 51,
 32,
 42,
 537,
 5,
 1122,
 3,
 2748,
 2317,
 238,
 3,
 324,
 5556,
 73,
 9,
 16,
 89,
 15,
 75,
 12765,
 5569,
 3,
 7,
 2,
 8027,
 159,
 3,
 63,
 150,
 3358,
 10381,
 3,
 7,
 2,
 29,
 235,
 2754,
 3,
 12620,
 15,
 3,
 9,
 2,
 32,
 7,
 5,
 180,
 2,
 8027,
 159,
 13091,
 23,
 9,
 1289,
 21628,
 1320,
 32,
 20,
 3,
 7,
 2,
 8027,
 159,
 3,
 15,
 7,
 73,
 9,
 158,
 835,
 2,
 9,
 3,
 195,
 4711,
 6,
 3,
 195,
 9,
 11374,
 9,
 3,
 8694,
 2771,
 5,
 325,
 3,
 195,
 4711,
 12825,
 565,
 3,
 35,
 3,
 15,
 40,
 3,
 8076,
 291,
 5569,
 278,
 221,


## 3. Fine-tuning del modelo

### 3.1. Lectura del modelo

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### 3.2. Evaluación durante el entrenamiento

A continuación implementamos un conjunto de funciones auxiliares para evluar los resultados durante el proceso de entrenamiento

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metrica de evaluación
metric = evaluate.load("rouge")

# Funciona auxiliar para preprocesar el texto
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum espera una nueva línea después de cada frase
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Reemplazamos -100 en las etiquetas porque no podemos decodificarlo
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Preprocesamos el texto
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 3.3. Lectura y adaptación de los datos para el entrenamiento

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Ignoramos los tokens relacionados con el padding durante el proceso de entrenamiento para los prompts
label_pad_token_id = -100

# Recolector de datos para el entrenamiento del modelo
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


### Preparación y ejecución del fine-tuning (entrenamiento)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"


REPOSITORY="/content/drive/MyDrive/flan-t5-small-fine-tuned"

# Definimos las opciones del entrenamiento
training_args = Seq2SeqTrainingArguments(
    # Hiperprámetros del entrenamiento
    output_dir=REPOSITORY,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=4,
    # Estrategias de logging y evaluación
    logging_dir=f"{REPOSITORY}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Creamos la instancia de entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_tokens["train"],
    eval_dataset=ds_tokens["validation"],
    compute_metrics=compute_metrics,
    processing_class="BatchEncoding"  # Aquí usamos el nuevo parámetro
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{REPOSITORY}/tokenizer")

('/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/spiece.model',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/flan-t5-small-fine-tuned/tokenizer/tokenizer.json')

In [ ]:
# Iniciamos el entrenamiento
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.620949,41.139600,26.953000,41.129200,41.220800,9.461538
2,No log,1.528900,42.982300,28.362100,42.791800,42.977000,9.785714
3,No log,1.477909,43.510900,28.669400,43.395600,43.519000,9.857143
4,No log,1.465194,43.919500,28.669400,43.729600,43.930000,9.895604


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now depre

TrainOutput(global_step=352, training_loss=1.932643716985529, metrics={'train_runtime': 196.5706, 'train_samples_per_second': 14.244, 'train_steps_per_second': 1.791, 'total_flos': 520493413171200.0, 'train_loss': 1.932643716985529, 'epoch': 4.0})

## 4. Generación de texto con Flan-T5 Fine-tuned y evaluación

### Lectura del modelo y del tokenizador

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

REPOSITORY="/content/drive/MyDrive/flan-t5-small-fine-tuned"

# Importamos el tokenizador
tokenizer_FT5_FT = T5Tokenizer.from_pretrained(f"{REPOSITORY}/tokenizer")

# Importamos el modelo con fine-tuning
model_FT5_FT = T5ForConditionalGeneration.from_pretrained(f"{REPOSITORY}/checkpoint-264", device_map="auto")

### Generación de texto

In [ ]:
context = """
    Sudoración \
    Escalofríos y temblores \
    Dolor de cabeza \
    Dolores musculares \
    Pérdida de apetito \
    Irritabilidad \
    Deshidratación \
    Debilidad general \
"""
# Fiebre

In [ ]:
context = """
Manchado o sangrado vaginal
Dolor o calambres en el abdomen o la parte inferior de la espalda
Fluidos o tejidos que salen por la vagina
Si ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.
Recuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.
"""

In [ ]:
context = """"
    Dolor de cabeza. \
    Tos persistente y seca. \
    Falta de aire. \
    Cansancio y debilidad. \
    Congestión o goteo nasal. \
    Dolor de garganta. \
    Dolor en los ojos. \
"""
# gripe(influencia)

In [ ]:
question = '¿Qué enfermedad tengo respecto a estos síntomas?'

In [ ]:
# Construimos el prompt conforme a la plantilla de fine-tuning
prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

# Transformamos los tokens generados en texto
print(tokenizer_FT5_FT.decode(outputs[0]))

<pad> Mujeres</s>


### Evaluación con el subconjunto de pruebas

In [ ]:
import torch

# Cambiamos el modelo al modo de evaluación
model_FT5_FT.eval()

# Definir tamaño del lote
batch_size = 8

all_predictions = []

# Deshabilitamos el entrenamiento y obtenemos las completions
with torch.no_grad():
  for i in range(0, len(ds_tokens["test"]["input_ids"]), batch_size):
        # Extraemos el lote actual
        input_ids_batch = torch.tensor(ds_tokens["test"]["input_ids"][i:i+batch_size], device='cuda:0')

        # Obtenemos las predicciones del modelo
        outputs = model_FT5_FT.generate(input_ids_batch)

        # Concatenemos las predicciones, convirtiéndolas a listas de Python
        all_predictions.extend([pred.cpu().tolist() for pred in outputs])

# Calculamos las metricas
labels = np.array(ds_tokens['test']['labels'])
# Pad the predictions to the maximum length
max_len = max(len(pred) for pred in all_predictions)
completions = np.array([pred + [tokenizer.pad_token_id] * (max_len - len(pred)) for pred in all_predictions])


metrics = compute_metrics((completions, labels))

print(metrics)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 38.6651, 'rouge2': 22.1992, 'rougeL': 38.4462, 'rougeLsum': 38.5173, 'gen_len': 9.882352941176471}
